# <font color='navy'> Nurse Scheduling Problem
## <font color='navy'> OR-Tools Approach

### <font color='navy'>Libraries

In [1]:
from ortools.sat.python import cp_model

### <font color='navy'>Problem proposal

#### <font color='navy'> Variables
- Nurses: $n \in N = \{1...5\}$
- Shifts: $s \in S = \{1...3\}$
- Days: $d \in D = \{1, 7\}$

In [2]:
num_nurses = 5
num_shifts = 3
num_days = 7
all_nurses = range(num_nurses)
all_shifts = range(num_shifts)
all_days = range(num_days)

#### <font color='navy'>Objective function and constraints:

- Each shift is assigned to a single nurse per day:
$$\sum^{}_{n \in N}\, X_{nsd} = 1\;\;\;\;\;\;\;\;\;\;\;\;(1)$$
    
- Each nurse must work at most one shift per day:
$$\sum^{}_{s \in S}\, X_{nsd} \leq 1\;\;\;\;\;\;\;\;\;\;\;\;(2)$$

- Each nurse works at least 2 shifts per week:
$$\sum^{}_{d \in D}\,\sum^{}_{s \in S}\, X_{nsd} \geq 2\;\;\;\;\;\;\;\;\;\;\;\;(3)$$

- Each nurse must not work consecutive shifts:
$$X_{i,3,t} + X_{i,1,t+1} \leq 1\;\;\;\;\;\;\;\;\;\;\;\;(4)$$

- **Objective function:** minimize the total number of weekly shifts worked by each nurse.

$$Minimize \sum_{n,s,d}{X_{n,s,d}}$$

In [5]:
model = cp_model.CpModel()

shifts = {}
for n in all_nurses:
    for d in all_days:
        for s in all_shifts:
            shifts[(n, d,
                    s)] = model.NewBoolVar('shift_n%id%is%i' % (n, d, s))

# There must be at least 1 nurse on each shift:
for d in all_days:
    for s in all_shifts:
        model.AddExactlyOne(shifts[(n, d, s)] for n in all_nurses)

# Every nurse must not work more than 1 shift per day:
for n in all_nurses:
    for d in all_days:
        model.AddAtMostOne(shifts[(n, d, s)] for s in all_shifts)
    
# Every nurse must work at least 2 shifts per week:
model.Add(sum(shifts[(n, d, s)]for s in all_shifts for d in all_days) >= 2)
        
# Distribute the shifts evenly:
min_shifts_per_nurse = (num_shifts * num_days) // num_nurses
if num_shifts * num_days % num_nurses == 0:
    max_shifts_per_nurse = min_shifts_per_nurse
else:
    max_shifts_per_nurse = min_shifts_per_nurse + 1
for n in all_nurses:
    num_shifts_worked = []
    for d in all_days:
        for s in all_shifts:
            num_shifts_worked.append(shifts[(n, d, s)])
    model.Add(min_shifts_per_nurse <= sum(num_shifts_worked))
    model.Add(sum(num_shifts_worked) <= max_shifts_per_nurse)
    
model.Minimize(sum(shifts[(n, d, s)] for n in all_nurses for s in all_shifts for d in all_days))

### <font color='navy'>Solver

In [7]:
solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0

# Enumerate all solutions:
solver.parameters.enumerate_all_solutions = True

In [8]:
class NursesPartialSolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, shifts, num_nurses, num_days, num_shifts, limit):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._shifts = shifts
        self._num_nurses = num_nurses
        self._num_days = num_days
        self._num_shifts = num_shifts
        self._solution_count = 0
        self._solution_limit = limit

    def on_solution_callback(self):
        self._solution_count += 1
        print('\nSolution %i' % self._solution_count)
        for d in range(self._num_days):
            print('Day %i' % int(d+1))
            for n in range(self._num_nurses):
                is_working = False
                for s in range(self._num_shifts):
                    if self.Value(self._shifts[(n, d, s)]):
                        is_working = True
                        print('  Nurse %i works shift %i' % (int(n+1), int(s+1)))
        if self._solution_count >= self._solution_limit:
            print('Stop search after %i solutions' % self._solution_limit)
            self.StopSearch()

    def solution_count(self):
        return self._solution_count

### <font color='navy'>Display Results

In [9]:
solution_limit = 3
solution_printer = NursesPartialSolutionPrinter(shifts, num_nurses,
                                                num_days, num_shifts,
                                                solution_limit)

solver.Solve(model, solution_printer)


Solution 1
Day 1
  Nurse 2 works shift 1
  Nurse 4 works shift 3
  Nurse 5 works shift 2
Day 2
  Nurse 3 works shift 1
  Nurse 4 works shift 3
  Nurse 5 works shift 2
Day 3
  Nurse 3 works shift 1
  Nurse 4 works shift 2
  Nurse 5 works shift 3
Day 4
  Nurse 1 works shift 3
  Nurse 4 works shift 2
  Nurse 5 works shift 1
Day 5
  Nurse 1 works shift 1
  Nurse 2 works shift 3
  Nurse 3 works shift 2
Day 6
  Nurse 1 works shift 1
  Nurse 2 works shift 2
  Nurse 3 works shift 3
Day 7
  Nurse 1 works shift 2
  Nurse 2 works shift 3
  Nurse 3 works shift 1


4